# Preparing Single station data
## using numpy and pandas libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
xl = pd.ExcelFile("./raw_data/NWM_data_updated.xlsx", engine='openpyxl')
a=xl.sheet_names



In [2]:
#loading the data set
dataframe_collection = {}
i=0
for i in range(4):
    if i==0:
        dataframe_collection[i] = xl.parse(a[i],converters={"STAID_1":str})
    else:
        dataframe_collection[i] = xl.parse(a[i])

obsflow = dataframe_collection[2]
modelflow= dataframe_collection[13]



In [3]:

j=15
k=0
column_names = ["dis2sea","landUse","soilTexture1", "soilTexture2","soilTexture3","soilTexture4","soilTexture5","soilTexture6","soilTexture7","soilTexture8","soilTexture9","soilTexture10","topoIndex","drainageArea","ModelFlow","Obsflow"]
df = pd.DataFrame(columns = column_names)  
for index, row in dataframe_collection[0].iterrows():
    print(index)
    sta_id = dataframe_collection[0].iloc[index,0]
    dis2sea=dataframe_collection[0].iloc[index,1]
    land_use=dataframe_collection[0].iloc[index,2]
    soil_texture1=dataframe_collection[0].iloc[index,3]
    soil_texture2=dataframe_collection[0].iloc[index,4]
    soil_texture3=dataframe_collection[0].iloc[index,5]
    soil_texture4=dataframe_collection[0].iloc[index,6]
    soil_texture5=dataframe_collection[0].iloc[index,7]
    soil_texture6=dataframe_collection[0].iloc[index,8]
    soil_texture7=dataframe_collection[0].iloc[index,9]
    soil_texture8=dataframe_collection[0].iloc[index,10]
    soil_texture9=dataframe_collection[0].iloc[index,11]
    soil_texture10=dataframe_collection[0].iloc[index,12]
    topoindex = dataframe_collection[0].iloc[index,13]
    drainagearea =  dataframe_collection[0].iloc[index,14]
    obs = obsflow[sta_id]/drainagearea
    mod = modelflow[sta_id]/drainagearea

    for i in range(979):
        df.loc[k]=[dis2sea]+[land_use]+[soil_texture1]+[soil_texture2]+[soil_texture3]+[soil_texture4]+[soil_texture5]+[soil_texture6]+[soil_texture7]+[soil_texture8]+[soil_texture9]+[soil_texture10]+[topoindex]+[drainagearea]+[mod[i]]+[obs[i]]
        k=k+1


0


# Saving the perpared data

In [4]:
df4 =df.dropna()
df4
df4.to_csv('out_10day_lead_data_single.csv', sep='\t')

# Reading data and the standardizing variables

In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv('out_10day_lead_data_single.csv',delimiter = '\t')
x_mean = pd.read_csv("./raw_data/NWM_ML_Scripts/x_mean.csv", sep='\t')
x_std =  pd.read_csv("./raw_data/x_std.csv", sep='\t')
y_mean =  pd.read_csv("./raw_data/y_mean.csv", sep='\t')
y_std =  pd.read_csv("./raw_data/y_std.csv", sep='\t')

In [6]:
x, y = df.iloc[:, 1:16], df.iloc[:,16]
l=10

In [7]:
d_area_x = x.iloc[:,13]
x-= x_mean.iloc[l,1:16]
x /= x_std.iloc[l,1:16]


# Using keras Load_model library component to load the trained model

In [1]:
from keras.models import load_model
model = load_model('./temporal_DLmodels/my_model_10_day.h5')
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])


# Predicting the results

In [9]:
z=model.predict(x)
l=10

x=x*x_std.iloc[l,1:16]
x=x +x_mean.iloc[l,1:16]
            

x.iloc[:,14]=x.iloc[:,14]*d_area_x


y=y*d_area_x

z = y_std.iloc[l,1]*z
z= z+y_mean.iloc[l,1]

d_area_x=np.array(d_area_x)
d_area_x.shape = [d_area_x.shape[0],1]
z=z*d_area_x


d_x= pd.DataFrame(x.iloc[:,14])
d_y=pd.DataFrame(y)
d_z=pd.DataFrame(z)
d_xy=pd.concat([d_x, d_y,d_z], axis=1)
d_xy.columns = ['Modelflow', 'Obsflow','Error']
d_xy



,Modelflow,Obsflow,Error
0,232.436870,396.435858,166.095631
1,212.933745,538.020093,148.804410
2,203.080620,509.703246,138.758597
3,186.608121,574.831994,121.295108
4,192.833746,512.534931,128.057463
...,...,...,...
849,109.151248,112.134714,55.724151
850,99.749998,101.657481,49.550283
851,96.356248,102.223818,47.014818
852,96.449998,94.011932,47.085913


# Saving the results to a file

In [10]:
d_xy.to_csv("./graph_data/Error_WholeData_abs.csv")

# DL Model Summary

In [2]:
model.summary()

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_240 (Dense)           (None, 512)               8192      
                                                                 
 dense_241 (Dense)           (None, 256)               131328    
                                                                 
 dense_242 (Dense)           (None, 64)                16448     
                                                                 
 dense_243 (Dense)           (None, 32)                2080      
                                                                 
 dense_244 (Dense)           (None, 16)                528       
                                                                 
 dense_245 (Dense)           (None, 1)                 17        
                                                                 
Total params: 158,593
Trainable params: 158,593
Non-t